# Find Possible Words from Phonemes

Created by Kamaljit Grewal. March 2024.

This is a solution to the following question: Given a sequence of phonemes as input (e.g. ["DH", "EH", "R", "DH", "EH", "R"]), find all the combinations of the words that can produce this sequence (e.g. [["THEIR", "THEIR"], ["THEIR", "THERE"], ["THERE", "THEIR"], ["THERE", "THERE"]]). You can preprocess the dictionary into a different data structure if needed.

This code utilizes Carnegie Mellon Univeristy's pronouncing dictionary as well as some starter code from kaggle user snimrexzeo (with edits from me) to perform data leaning.

## Data Cleaning

In [1]:
import pandas as pd

# We will create a dataframe (dictionary) where each word maps to a list of phonemes.

words = []
phonemes_list = []
dict_file = open('../input/cmu-pronouncing-dictionary/cmudict.dict', 'r')

line = [line.rstrip('\n') for line in dict_file]
for l in line:
    text = l.split(' ')
    words.append(text[0]) #first part of a line is the word
    phonemes_list.append(' '.join(text[1:])) #the rest is phonemes
    
data = pd.DataFrame({'Word': words, 'Phonemes': phonemes_list})

data

,Word,Phonemes
0,'bout,B AW1 T
1,'cause,K AH0 Z
2,'course,K AO1 R S
3,'cuse,K Y UW1 Z
4,'em,AH0 M
...,...,...
135005,zyuganov,Z Y UW1 G AA0 N AA0 V
135006,zyuganov(2),Z UW1 G AA0 N AA0 V
135007,zyuganov's,Z Y UW1 G AA0 N AA0 V Z
135008,zyuganov's(2),Z UW1 G AA0 N AA0 V Z


In [2]:
indices_to_drop = []

# isaplha means aplhabet letters a-z. This means drop rows whose values aren't
# purely comprised of alphabet letters a-z.
for i, row in data.iterrows():
    for j in range(len(row['Word'])):
        if not row['Word'][j].isalpha(): 
            indices_to_drop.append(i)
            
# I added this: remove the numbers from the phonemes (i.e., DH EH1 R becomes DH EH R)
for i, row in data.iterrows():
    phonemes = row['Phonemes']
    phonemes_without_numbers = ''.join(char if char.isalpha() or char == ' ' else '' for char in phonemes)
    data.at[i, 'Phonemes'] = phonemes_without_numbers
            
# Drop the rows outside the loop to avoid modifying the DataFrame while iterating
data = data.drop(index=indices_to_drop)

# Reset the index after dropping rows
data = data.reset_index(drop=True)

data

,Word,Phonemes
0,a,AH
1,aaa,T R IH P AH L EY
2,aaberg,AA B ER G
3,aachen,AA K AH N
4,aachener,AA K AH N ER
...,...,...
117384,zynda,Z IH N D AH
117385,zysk,Z AY S K
117386,zyskowski,Z IH S K AO F S K IY
117387,zyuganov,Z Y UW G AA N AA V


# Finding Word Combos

In [3]:
from collections import defaultdict
from typing import Sequence

def find_word_combos_with_pronunciation(phonemes: Sequence[str]) -> Sequence[Sequence[str]]:
    def backtrack(start_index, path):
        if start_index == len(phonemes): # at the end of the phonemes sequence. return what you've got.
            result.append(path[:])
            return
        for end_index in range(start_index + 1, len(phonemes) + 1):
            current_phonemes = ' '.join(phonemes[start_index:end_index])
            
            # See if our phoneme matches anything in the Phonemes column of the dataframe
            Series = data.loc[data['Phonemes'] == current_phonemes]
            if not Series.empty:
                for i in Series.Word: # if we get a match, add the corresponding Word to path
                    path.append(i)
                    backtrack(end_index, path)
                    path.pop()
    
    result = [] # starts here
    backtrack(0, [])
    return result

# Example usage
phonemes_sequence = ["DH", "EH", "R", "DH","EH","R"]
result = find_word_combos_with_pronunciation(phonemes_sequence)
print(result)

phonemes_sequence = ["S", "AH", "M", "W", "AH", "N"]
result = find_word_combos_with_pronunciation(phonemes_sequence)
print(result)

[['their', 'their'], ['their', 'there'], ['there', 'their'], ['there', 'there']]
[['suh', 'mm', 'one'], ['suh', 'mm', 'won'], ['some', 'one'], ['some', 'won'], ['sum', 'one'], ['sum', 'won'], ['someone']]
